# Chapter 11: Training Deep Neural Nets

In the previous chapter, we trained a neural network with 2 hidden layers. More complex problems require networks with more hidden layers with hundreds of neurons per layer. Training these can lead to several problems:

- The _vanishing gradients_ and _exploding gradients_ problem makes lower levels hard to train.
- Training a large network can be very slow.
- A model with millions of parameters risks overfitting the training data.

Below we will discuss methods for solving all of these problems.

## Vanishing/Exploding Gradients Problem

While training a neural network with backpropagation, the algorithm finds the components of the error contributed by each layer to compute the error gradient.  Gradients can often get smaller and smaller as the algorithm progresses, resulting in the gradient contribution from the lower layers approaching zero. This is known as the _vanishing gradient_ problem. Alternatively, the gradient can also can grow bigger and bigger which can cause the algorithm to diverge. This is called the _exploding gradient problem_.

Around 2010, a paper titled ["Understanding the Difficulty of Training Deep Feedforward Neural Networks"](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) found some reasons for this. The sigmoid activation function as well as the random initialization of the weight matrices' elements using a normal distribution with a mean of 0 and a standard deviation of 1. The paper showed the variance of the outputs was much larger than the variance of the inputs. Going forward in the network, the variance kept getting larger and it results in the activation saturating near the horizontal asymptotes, which causes the gradient to vanish.

### Xavier and He Initialization

The authors of the paper found that one way to prevent the vanishing/exploding gradient problem is to ensure that the variance of the input and output of each layer is the same. One way to do this is to initialize the weights matrix using a normal distribution with a mean of 0 and a standard deviation given by

$$ \sigma = \sqrt{\frac{2}{n_\text{ inputs} + n_\text{ outputs}}} $$

or a uniform distribution centered at 0 with a radius, $r$, given by

$$ r = \sqrt{\frac{6}{n_\text{ inputs} + n_\text{ outputs}}} $$

where $n_\text{ inputs}$ and $n_\text{ outputs}$ is the number of input  or output connections in that particular layer. This is often known as _Xavier initialization_ after the author's first name, or sometimes _Glorot initialization_.

For the ReLU activation function, we use a normal distribution with a standard deviation given by

$$ \sigma = \frac{2}{\sqrt{n_\text{ inputs} + n_\text{ outputs}}} $$

or a uniform distribution with a radius given by

$$ r = \sqrt{\frac{24}{n_\text{ inputs} + n_\text{ outputs}}} $$

which is known as _He initialization_. Below is an example of creating a layer of a neural network which uses _He initialization_. By default, `tf.layers.dense()` uses Xavier initialization.

In [0]:
import tensorflow as tf

n_inputs = 28 ** 2
n_hidden = 100

X = tf.placeholder(tf.float32, shape=(None, n_inputs))
he_init = tf.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden, activation=tf.nn.relu,
                          kernel_initializer=he_init, name='hidden1')

### Nonsaturating Activation Functions

One of the causes of the vanishing/exploding gradient problem discussed in the paper is the sigmoid activation function. The ReLU activation function performs much better, but it has a different problem. If some neurons output negative values, after the application of the activation function, their output will be stuck at 0. Since the gradient is also 0, the neuron remains "dead."

One solution to this problem is to use a "leaky" ReLU function, given by

$$ \text{LeakyReLU}(z) = \max(\alpha z, z) $$

where $\alpha$ is the slope ofthe ReLU function when the value of $z$ is less than 0. Researchers have found that this activation function performs better than the "hard" ReLU function. You can even have $\alpha$ be a parameter that the model learns during training. This prevents neurons from completely dying.

Another activation function that performs better than leaky ReLU that was proposed in this [paper](https://arxiv.org/pdf/1511.07289v5.pdf) by Djork-Arné Clevert called the _exponential linear unit_ (ELU) given by

$$ \text{ELU}_\alpha(z) = \left\{ \begin{matrix}
\alpha\,(\exp(z) - 1) && \text{if}\;z < 0 \\
z && \text{if}\; z \geq 0
\end{matrix} \right. $$

It has the following differences from the ReLU function:

- It takes negative values when $z < 0$ . which allows the unit to have an average output closer to 0. This helps alleviate the vanishing radient problem. You can tweak the hyperparameter, $\alpha$, sets the negative number that ELU approaches.

- It has a nonzero gradient when $z < 0$, preventing the dying units issue.

- The function is differentiable everywhere, which helps the speed of Gradient Descent.

The disadvantage of ELU is that it takes longer to compute than ReLU. The extra time is compensated for the fact that it helps Gradient Descent converge fasted, but it does cause the model to make predictions more slowly.

TensorFlow offeres an implementation of ELU which is used in the code example below:

In [0]:
tf.reset_default_graph()

n_inputs = 28 ** 2
n_hidden = 100

X = tf.placeholder(tf.float32, shape=(None, n_inputs))
he_init = tf.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, name='hidden1')

TensorFlow does not have an implementation of leaky ReLU, but it is easy to define ourselves:

In [0]:
tf.reset_default_graph()

def leaky_relu(z, alpha=0.01):
  return tf.maximum(alpha * z, z)

n_inputs = 28 ** 2
n_hidden = 100

X = tf.placeholder(tf.float32, shape=(None, n_inputs))
he_init = tf.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden, activation=leaky_relu, name='hidden1')

### Batch Normalization

In this [paper](https://arxiv.org/pdf/1502.03167v3.pdf) Sergey Ioffe and Christian Szegedy proposed a technique called _Batch Normalization_ (BN) to address both the vanishing/exploding gradient problem and the problem that the distribution of each layer's inputs change when the parameters of the previous layers change (i.e. the _Internal Covariate Shift_ problem).

The technique adds an operation to the model just before applying the activation function of each layer. It zero-centers and normalizes the inputs, then it scales and shifts the result using two new parameters per layer. This lets the model learn the optimal mean and shift for each layer.

The algorithm starts by first computing the empirical mean for the current mini-batch, $B$, given by

$$ \mu_B = \frac{1}{m_B} \sum\limits_{i\,=\,1}^{m_B} \mathbf{x}^{(i)} $$

Next, we find the empirical standard deviation, given by

$$ \sigma_B^{\;\;2} = \frac{1}{m_B} \sum\limits_{i\,=\,1}^{m_B} \left( \mathbf{x}^{(i)} - \mu_B \right)^2 $$

Then we zero-center and normalize the inputs in the mini-batch

$$ \hat{\mathbf{x}}^{(i)} = \frac{\mathbf{x}^{(i)} - \mu_B}{\sqrt{\sigma_B^{;\;2} + \epsilon}} $$

where $\epsilon$ is a small number, typically $10^{-5}$, called the _smoothing term_ to avoid division by zero. Finally it computes the output given by

$$ \mathbf{z}^{(i)} = \gamma\,\hat{\mathbf{x}}^{(i)} + \beta $$

where $\gamma$ is the scaling parameter and $\beta$ is the shift parameter which are learned during training.

When the model makes predictions, it uses the empirical mean and standard deviation of the entire training set. In the end, the model ends up learning 4 parameters: the mean of the training set, $\mu$; the standard deviation of the training set, $\sigma$; the scaling parameter, $\gamma$; and the shift parameter, $\beta$.

Adding Batch Normalization to a deep neural network improves the performance of the model, lets you skip normalizing the data before training the data, and helps the model converge to the optimal parameters in fewer training iterations. However, using Batch Normalization causes the model to make predictions slower since it adds another computational step for making predictions.

#### Implementing Batch Normalization with TensorFlow

TensorFlow provides a `tf.nn.batch_normalization()` function which normalizes and centers the data, but you must compute the mean and standard deviation yourself.You also have to handle the creation of the scaling and offfset parameters. TensorFlow also includes a `tf.layers.batch_normalization()` function which handles all of batch normalization for you. Below is an example.

In [0]:
tf.reset_default_graph()

n_inputs = 28 ** 2 # MNIST dataset.
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')

# Indicates if the batch normalization should be using the mini-batch's mean
# or the mean of the entire training set (same with standard deviation).
training = tf.placeholder_with_default(False, shape=(), name='training')

hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1')
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name='hidden2')
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name='outputs')
logts = tf.layers.batch_normalization(logits_before_bn, training=training,
                                      momentum=0.9)

The BN algorithm uses _exponential decay_ to compute a running average, which is why it requires the _momentum_ parameter. Given a new value, $v$, it updates the running average $\hat{v}$ given by

$$ \hat{v} \leftarrow \hat{v} \times \text{momentum} + v \times (1 - \text{momentum}) $$

Momentum values should be typically close to 1, e.g. 0.9, 0.99, or 0.999.

Below is an example of using _partial application_ using the `functools` library in order to make the code less repetitive.

In [0]:
from functools import partial

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')

# Indicates if the batch normalization should be using the mini-batch's mean
# or the mean of the entire training set (same with standard deviation).
training = tf.placeholder_with_default(False, shape=(), name='training')

batch_norm_layer = partial(tf.layers.batch_normalization,
                           training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1')
bn1 = batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name='hidden2')
bn2 = batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name='outputs')
logits = batch_norm_layer(logits_before_bn)

In [0]:
# Setting up the rest of the graph for training.

y = tf.placeholder(tf.int32, shape=(None), name='y')

x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                           labels=y)
loss = tf.reduce_mean(x_entropy, name='loss')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [0]:
# Downloading the MNIST dataset.

import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

def shuffle_batch(X, y, batch_size):
  rnd_idx = np.random.permutation(len(X))
  n_batches = len(X) // batch_size
  for batch_idx in np.array_split(rnd_idx, n_batches):
    X_batch, y_batch = X[batch_idx], y[batch_idx]
    yield X_batch, y_batch

In [42]:
# Training the neural network using Batch Normalization.
# In just 20 training iterations it achieves 97% accuracy on the
# validation set.

n_epochs = 20
batch_size = 200

# These extra ops are for training the scaling and offset parameters
# in batch normalization.
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
      sess.run([training_op, extra_update_ops],
               feed_dict={training: True, X: X_batch, y: y_batch})
    accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    print('Epoch: {} Validation Accuracy: {}'.format(epoch, accuracy_val))

Epoch: 0 Validation Accuracy: 0.8845999836921692
Epoch: 1 Validation Accuracy: 0.9092000126838684
Epoch: 2 Validation Accuracy: 0.9232000112533569
Epoch: 3 Validation Accuracy: 0.9301999807357788
Epoch: 4 Validation Accuracy: 0.9386000037193298
Epoch: 5 Validation Accuracy: 0.9422000050544739
Epoch: 6 Validation Accuracy: 0.9452000260353088
Epoch: 7 Validation Accuracy: 0.9506000280380249
Epoch: 8 Validation Accuracy: 0.9538000226020813
Epoch: 9 Validation Accuracy: 0.9557999968528748
Epoch: 10 Validation Accuracy: 0.9595999717712402
Epoch: 11 Validation Accuracy: 0.9602000117301941
Epoch: 12 Validation Accuracy: 0.9624000191688538
Epoch: 13 Validation Accuracy: 0.9642000198364258
Epoch: 14 Validation Accuracy: 0.9648000001907349
Epoch: 15 Validation Accuracy: 0.9682000279426575
Epoch: 16 Validation Accuracy: 0.9675999879837036
Epoch: 17 Validation Accuracy: 0.9679999947547913
Epoch: 18 Validation Accuracy: 0.9679999947547913
Epoch: 19 Validation Accuracy: 0.9706000089645386


An alternate syntax to training a model this way is to define the `training_op` the following way:

```python
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        training_op = optimizer.minimize(loss)
```

this lets you train the model using the more simple syntax:

```python
sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
```

### Gradient Clipping

One way to solve the exploding gradients problem is to clip the gradients' values to a defined range. This technique is called [_Gradient Clipping_](http://proceedings.mlr.press/v28/pascanu13.pdf). Though in general people prefer Batch Normalization. Below is an example of Gradient Clipping using TensorFlow:

In [0]:
# An example of gradient clipping.

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

training = tf.placeholder_with_default(False, shape=(), name='training')

batch_norm_layer = partial(tf.layers.batch_normalization,
                           training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1')
hidden1_act = tf.nn.elu(hidden1)

hidden2 = tf.layers.dense(hidden1_act, n_hidden2, name='hidden2')
hidden2_act = tf.nn.elu(hidden2)

logits_before_bn = tf.layers.dense(hidden2_act, n_outputs, name='outputs')
logits = batch_norm_layer(logits_before_bn)

x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                           labels=y)
loss = tf.reduce_mean(x_entropy, name='loss')

# Gradient clipping is here.
threshold = 1.0
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

## Reusing Pretrained Layers

Instead of training a large DNN from scratch, it is generally better to reuse an existing neural network used for a similar task, then reuse the lower layers of this network. This technique is called _transfer learning_.

### Reusing a TensorFlow model

Below is an example of saving and restoring a TensorFlow model using the `tr.train.import_meta_graph()` function:

In [0]:
# Defining the graph and saving it.

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

training = tf.placeholder_with_default(False, shape=(), name='training')

batch_norm_layer = partial(tf.layers.batch_normalization,
                           training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1')
bn1 = batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name='hidden2')
bn2 = batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name='outputs')
logits = batch_norm_layer(logits_before_bn)

training = tf.placeholder_with_default(False, shape=(), name='training')

x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                           labels=y)
loss = tf.reduce_mean(x_entropy, name='loss')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

init = tf.global_variables_initializer()

# New code here!
saver = tf.train.Saver()
with tf.Session() as sess:
  init.run()
  saver.save(sess, './my_model.ckpt')

In [0]:
# Getting nodes in the previous graph for reuse.

X = tf.get_default_graph().get_tensor_by_name('X:0')
y = tf.get_default_graph().get_tensor_by_name('y:0')
accuracy = tf.get_default_graph().get_tensor_by_name('accuracy:0')
training_op = tf.get_default_graph().get_operation_by_name('GradientDescent')

In [57]:
# Listing the operations in the predefined graph, truncated for readability.

for op in tf.get_default_graph().get_operations()[:20]:
  print(op.name)

X
y
training/input
training
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
hidden1/MatMul
hidden1/BiasAdd


Below is an example of creating a collection of important operations. This is often helpful if the graph is large and you only want to reuse certain operations.

In [0]:
# Defining the original graph.

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

training = tf.placeholder_with_default(False, shape=(), name='training')

batch_norm_layer = partial(tf.layers.batch_normalization,
                           training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1')
bn1 = batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name='hidden2')
bn2 = batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name='outputs')
logits = batch_norm_layer(logits_before_bn)

training = tf.placeholder_with_default(False, shape=(), name='training')

x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                           labels=y)
loss = tf.reduce_mean(x_entropy, name='loss')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

init = tf.global_variables_initializer()

# New code here!
for op in (X, y, accuracy, training_op):
  tf.add_to_collection('my_important_ops', op)

saver = tf.train.Saver()
with tf.Session() as sess:
  init.run()
  saver.save(sess, './my_model.ckpt')

In [0]:
# Restoring the graph and getting the operations from the collection.

tf.reset_default_graph()

saver = tf.train.import_meta_graph('./my_model.ckpt.meta')

X, y, accuracy, training_op = tf.get_collection('my_important_ops')

You can also define a restore Saver which will only restore specified variables. This is useful if you only want to restore the lower layers of a neural network. Below is an example of restoring only the lower layers of a neural network using a saver which only restores specified variables.

In [0]:
# Defining a graph with 5 hidden layers. First implementing gradient clipping.

tf.reset_default_graph()

n_inputs = 28 ** 2

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None,), name='y')

training = tf.placeholder_with_default(False, shape=(None))

with tf.name_scope('dnn'):
  layer = X
  for i, n_hidden in enumerate((300, 100, 50, 20)):
    hidden = tf.layers.dense(layer, n_hidden1,
                             name='hidden{}'.format(i+1))
    layer = tf.nn.relu(hidden, name='relu{}'.format(i+1))
  logits = tf.layers.dense(layer, n_outputs, name='outputs')

with tf.name_scope('loss'):
  x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                             labels=y)
  loss = tf.reduce_mean(x_entropy, name='loss')

with tf.name_scope('train'):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
  training_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
  correct = tf.nn.in_top_k(logits, y, 1)
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

In [0]:
# Defining the savers and running the training.

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope='hidden[123]') # Regex string
reuse_saver = tf.train.Saver(reuse_vars)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [117]:
# Training the model one time to train the lower layers of the neural network.

import os

model_path = './my_model.ckpt'

with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
      sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
    accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    print('Epoch: {} Validation set accuracy: {}'.format(epoch, accuracy_val))
  saver.save(sess, model_path)

Epoch: 0 Validation set accuracy: 0.7983999848365784
Epoch: 1 Validation set accuracy: 0.8791999816894531
Epoch: 2 Validation set accuracy: 0.8996000289916992
Epoch: 3 Validation set accuracy: 0.9128000140190125
Epoch: 4 Validation set accuracy: 0.920199990272522
Epoch: 5 Validation set accuracy: 0.9265999794006348
Epoch: 6 Validation set accuracy: 0.930400013923645
Epoch: 7 Validation set accuracy: 0.9337999820709229
Epoch: 8 Validation set accuracy: 0.9381999969482422
Epoch: 9 Validation set accuracy: 0.9405999779701233
Epoch: 10 Validation set accuracy: 0.9426000118255615
Epoch: 11 Validation set accuracy: 0.9472000002861023
Epoch: 12 Validation set accuracy: 0.9484000205993652
Epoch: 13 Validation set accuracy: 0.9508000016212463
Epoch: 14 Validation set accuracy: 0.9527999758720398
Epoch: 15 Validation set accuracy: 0.954800009727478
Epoch: 16 Validation set accuracy: 0.9575999975204468
Epoch: 17 Validation set accuracy: 0.9584000110626221
Epoch: 18 Validation set accuracy: 0.9581

In [118]:
# Training the model again, this time we restore the lower layers

new_model_path = './my_model_new.ckpt'

with tf.Session() as sess:
  init.run()
  reuse_saver.restore(sess, model_path)
  for epoch in range(n_epochs):
    for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
      sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
    accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    print('Epoch: {} Validation set accuracy: {}'.format(epoch, accuracy_val))
  saver.save(sess, new_model_path)

INFO:tensorflow:Restoring parameters from ./my_model.ckpt
Epoch: 0 Validation set accuracy: 0.9381999969482422
Epoch: 1 Validation set accuracy: 0.9459999799728394
Epoch: 2 Validation set accuracy: 0.951200008392334
Epoch: 3 Validation set accuracy: 0.9531999826431274
Epoch: 4 Validation set accuracy: 0.9545999765396118
Epoch: 5 Validation set accuracy: 0.9567999839782715
Epoch: 6 Validation set accuracy: 0.9603999853134155
Epoch: 7 Validation set accuracy: 0.9603999853134155
Epoch: 8 Validation set accuracy: 0.9617999792098999
Epoch: 9 Validation set accuracy: 0.9634000062942505
Epoch: 10 Validation set accuracy: 0.9649999737739563
Epoch: 11 Validation set accuracy: 0.9646000266075134
Epoch: 12 Validation set accuracy: 0.9664000272750854
Epoch: 13 Validation set accuracy: 0.9661999940872192
Epoch: 14 Validation set accuracy: 0.9667999744415283
Epoch: 15 Validation set accuracy: 0.9688000082969666
Epoch: 16 Validation set accuracy: 0.9675999879837036
Epoch: 17 Validation set accuracy: 

### Reusing Models From another Frameworks

Below is an example to illustrate how to import models from other frameworks into TensorFlow. This code sets the kernel and bias of a hidden layer at the start of the TensorFlow session using the initializer.


In [121]:
tf.reset_default_graph()

n_inputs = 2
n_hidden = 3

original_w = [[1., 2., 3.], [4., 5., 6.]]
original_b = [7., 8., 9.]

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.relu, name='hidden')

graph = tf.get_default_graph()
assign_kernel = graph.get_operation_by_name('hidden/kernel/Assign')
assign_bias = graph.get_operation_by_name('hidden/bias/Assign')

init_kernel = assign_kernel.inputs[1]
init_bias = assign_bias.inputs[1]

init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init, feed_dict={
    init_kernel: original_w,
    init_bias: original_b,
  })
  print(hidden.eval(feed_dict={X: [[10., 11.]]}))

[[ 61.  83. 105.]]


Another way is to make dedicated nodes for assigning the hidden layer's kernel and bias, then set them at any point using placeholders. This is more verbose but allows more control.

In [122]:
tf.reset_default_graph()

n_inputs = 2
n_hidden = 3

original_w = [[1., 2., 3.], [4., 5., 6.]]
original_b = [7., 8., 9.]

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.relu, name='hidden')

with tf.variable_scope('', default_name='', reuse=True):
  hidden_weights = tf.get_variable('hidden/kernel')
  hidden_bias = tf.get_variable('hidden/bias')
  
original_weights = tf.placeholder(tf.float32, shape=(n_inputs, n_hidden))
original_bias = tf.placeholder(tf.float32, shape=(n_hidden))

assign_hidden_weights = tf.assign(hidden_weights, original_weights)
assign_hidden_bias = tf.assign(hidden_bias, original_bias)

init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  sess.run(assign_hidden_weights, feed_dict={original_weights: original_w})
  sess.run(assign_hidden_bias, feed_dict={original_bias: original_b})
  print(hidden.eval(feed_dict={X: [[10., 11.]]}))

[[ 61.  83. 105.]]
